**구현하고자 하는 기능:**
1. ~~지번을 입력받으면 해당 지번 물건의 반경 안에 있는 매물들을 모두 지도상에 표시한다~~
2. ~~매물들의 세부정보 popup에 추가~~
3. ~~해당 지번은 주변 매물들이 지도상에 표시가 된다.~~
4. ~~마우스를 올리면 매물에 대한 정보가 요약된다. ~~
5. ~~주변 지하철이 표시가 된다.~~
6. 주변 공원 및 큰 상점들이 표시가 된다.
7. 입력받은 지번에 해당하는 매물을 따로 분류하는 마커 로직 완성하기
8. 유사한 매물들만 표시되게 한다. -> 유사함을 정의(필) 

In [1]:
# 모듈, 경로 설정
import pandas as pd
import pydeck as pdk
import geopandas as gpda
import config
import json
import requests
import sys
import folium

path = "C:\github/aide-analysis/"
sys.path.append(path)

from src.sql import DatabaseHandler
db = DatabaseHandler()

from core.config import Config
config = Config()


In [2]:
# 경로설정
import sys
path = "C:\github/aide-analysis/"
sys.path.append(path)

# 모듈 import
import pandas as pd
import src
import folium
from src.openapi import KakaoLocalAPI as kakao
from openpyxl import load_workbook
from src.utils import insert_dataframe_to_excel_sheet
from src.sql import DatabaseHandler
from src.temp import *


db = DatabaseHandler()

# 카카오 API 호출
docs = kakao.search_address('서울시 서초구 반포동 20-43')


# 파싱정보
# 지번 추출
jibeon = docs['documents'][0]['address_name']

# 시도/ 시군구/ 행정동 코드 추출
sidoCode = docs["documents"][0]["address"]["h_code"][:2]
sigunguCode = docs["documents"][0]["address"]["h_code"][:5]
hdongCode = docs["documents"][0]["address"]["h_code"]

# 행정구역 명칭
sido_ = docs["documents"][0]["address"]["region_1depth_name"]
sigungu_ = docs["documents"][0]["address"]["region_2depth_name"]
if sigungu_ == "":
    sigungu_ = sido_
hdong_ = docs['documents'][0]['address']['region_3depth_h_name']
if hdong_ == "":
    hdong_ = docs['documents'][0]['address']['region_3depth_name']

# 경도, 위도
lon = docs['documents'][0]['address']['x'] 
lat = docs['documents'][0]['address']['y'] 
radius = 1000

# 단지명
danjiName = docs['documents'][0]['road_address']['building_name']

# 시계열 인자
startyear = datetime.now().year-10
endyear = datetime.now().year-1



In [3]:
docs

{'documents': [{'address': {'address_name': '서울 서초구 반포동 20-43',
    'b_code': '1165010700',
    'h_code': '1165056000',
    'main_address_no': '20',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '서초구',
    'region_3depth_h_name': '반포1동',
    'region_3depth_name': '반포동',
    'sub_address_no': '43',
    'x': '127.011034636758',
    'y': '37.5063518132638'},
   'address_name': '서울 서초구 반포동 20-43',
   'address_type': 'REGION_ADDR',
   'road_address': {'address_name': '서울 서초구 신반포로 270',
    'building_name': '반포자이아파트',
    'main_building_no': '270',
    'region_1depth_name': '서울',
    'region_2depth_name': '서초구',
    'region_3depth_name': '반포동',
    'road_name': '신반포로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.011582043737',
    'y': '37.5078602234885',
    'zone_no': '06544'},
   'x': '127.011034636758',
   'y': '37.5063518132638'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [5]:
# folium용
# 입력된지번의 일정 반경내 아파트와 아파트 정보
sql = f"""
WITH tmp AS (  SELECT A.pnuCode, 
                      addressJibeon, 
                      addressRoad, 
                      danjiName, 
                      usePermissionDate, 
                      B.contractYear, 
                      B.contractMonth, 
                      B.jeonseJeonyongUnitPriceAverage, 
                      buildingAreaPerLandArea, 
                      floorAreaRatio, 
                      mainPurposeCodeName, 
                      otherPurpose, 
                      saedaeCount, 
                      totalParkingLotCount, 
                      lon, 
                      lat, 
                      coord, 
                      geom
               FROM aide.apartment_danji_information AS A
               JOIN (SELECT T1.*
                     FROM(
                         SELECT pnuCode,
                                contractYear,
                                contractMonth,
                                supplyArea,
                                tradingJeonyongUnitPriceAverage,
                                jeonseJeonyongUnitPriceAverage,
                                wolseJeonyongUnitPriceAverage,
                                ROW_NUMBER() OVER(PARTITION BY pnuCode  ORDER BY contractYear DESC, contractMonth DESC) AS num
                         FROM aide.apartment_danji_monthly_area_price
                         WHERE pnuCode LIKE '{sidoCode}%' AND jeonseJeonyongUnitPriceAverage is not null
                        ) AS T1
                     WHERE T1.num = 1 
                   ) AS B
                   on A.pnuCode = B.pnuCode 
                   
)SELECT tmp.*,
        ST_Distance_Sphere(POINT({lon},{lat}),geom) AS distance
FROM tmp
WHERE ST_Distance_Sphere(POINT({lon},{lat}), geom) <={radius}
ORDER BY distance ASC;
"""
df= db.read_table(sql)


# 반경내 지하철역 정보 테이블
sql2 = f"""
SELECT * , 
       ST_Distance_Sphere(POINT({lon},{lat}),POINT(lon,lat)) AS distance
FROM aide.railway_station_info
WHERE ST_Distance_Sphere(POINT({lon},{lat}),POINT(lon,lat)) <= {radius}
ORDER BY distance ASC;
"""
train = db.read_table(sql2)

In [7]:
df

In [6]:
from IPython.display import IFrame
# params
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']
center = [37.5150660594, 127.0180350423]

# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 14,
    tiles = tiles[2]
)

#     text1=f"""지번:{df.iloc[i]['addressJibeon']} <br> 
#           단지명 :{df.iloc[i]['danjiName']} <br> 
#           사용승인일 :{df.iloc[i]['usePermissionDate']} <br> 
#           전세전용평단가평균 : {df.iloc[i]['jeonseJeonyongUnitPriceAverage']} <br> 
#           용적률 : {df.iloc[i]['floorAreaRatio']} <br>
#           주용도 :{df.iloc[i]['mainPurposeCodeName']} <br>
#           기타용도 : {df.iloc[i]['otherPurpose']} <br>
#           세대수 : {df.iloc[i]['saedaeCount']} <br>
#           총 주차대수 : {df.iloc[i]['totalParkingLotCount']} <br>
#           거리 : {df.iloc[i]['distance']}""" 


text2 = f"""지번:{df.iloc[0]['addressJibeon']} <br> 
          단지명 :{df.iloc[0]['danjiName']} <br> 
          사용승인일 :{df.iloc[0]['usePermissionDate']} <br> 
          전세전용평단가평균 : {df.iloc[0]['jeonseJeonyongUnitPriceAverage']} <br> 
          용적률 : {df.iloc[0]['floorAreaRatio']} <br>
          주용도 :{df.iloc[0]['mainPurposeCodeName']} <br>
          기타용도 : {df.iloc[0]['otherPurpose']} <br>
          세대수 : {df.iloc[0]['saedaeCount']} <br>
          총 주차대수 : {df.iloc[0]['totalParkingLotCount']} <br>
          거리 : {df.iloc[0]['distance']}""" 



# 입력받은 지번을 제외한 매물 마킹
for i in range(1,len(df)):
    folium.Marker(
        location = [df.iloc[i]['lat'], df.iloc[i]['lon']],      
        popup = folium.Popup(f"""지번:{df.iloc[i]['addressJibeon']} <br> 
                                 단지명 :{df.iloc[i]['danjiName']} <br> 
                                 사용승인일 :{df.iloc[i]['usePermissionDate']} <br> 
                                 전세전용평단가평균 : {df.iloc[i]['jeonseJeonyongUnitPriceAverage']} <br> 
                                 용적률 : {df.iloc[i]['floorAreaRatio']} <br>
                                 주용도 :{df.iloc[i]['mainPurposeCodeName']} <br>
                                 기타용도 : {df.iloc[i]['otherPurpose']} <br>
                                 세대수 : {df.iloc[i]['saedaeCount']} <br>
                                 총 주차대수 : {df.iloc[i]['totalParkingLotCount']} <br>
                                 거리 : {df.iloc[i]['distance']}""" , 
                             min_width =400,
                             max_width = 400),
        tooltip = df.iloc[i]['danjiName'],
        icon = folium.Icon('blue', icon = icon[1])).add_to(m)
    
# 입력받은 지번 매물 마킹
folium.Marker(
    location = [lat,lon],
    popup = folium.Popup(text2, min_width = 500, max_widht = 500),
    tooltip = danjiName,
    icon = folium.Icon('red', icon = icon[0])).add_to(m)
 
    
for i in range(0, len(train)):
    folium.Marker(
        location = [train.iloc[i]['lat'], train.iloc[i]['lon']],
        popup = train.iloc[i]['lineName'],
        tooltip = train.iloc[i]['lineName'],
        icon = folium.Icon('green', icon = icon[1])).add_to(m)
m    


AttributeError: 'NoneType' object has no attribute 'iloc'